In [1]:
import os
import json
import re
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader

In [2]:
def documents_transformation(input_dir: str):
        print(f"Input directory: {input_dir}")
        documents = SimpleDirectoryReader(input_dir=input_dir).load_data()
        transformed_documents = []
        for doc in documents:
            transformed_content = doc.get_content().lower()
            transformed_content = re.sub(r'\s+', ' ', transformed_content)
            transformed_content = re.sub(r'[^\w\s]', '', transformed_content)
            transformed_documents.append(Document(text=transformed_content, metadata=doc.metadata))
        return transformed_documents

In [ ]:
def process_documents(input_dir: str) -> None:
        input_dir = rf"{input_dir}"
        documents = documents_transformation(input_dir)
        print("Document Transformation is done")
 